In [5]:
import numpy as np
import pandas as pd
from pathlib import Path
import os
import yaml
import cv2
from copy import deepcopy

In [6]:
LABELS_PATH = Path("../data/Custom_data.yaml")
with open(LABELS_PATH, "r") as f:
    names = yaml.safe_load(f)["names"]

In [7]:
names = dict(enumerate(names))

In [8]:
TRAIN_LABELS_PATH = Path("../data/TrafficSignLocalizationandDetection/train/labels/")

In [9]:
def get_class(file):
    with open(file, "r") as f:
        l = f.readlines()
    return int(l[0].strip().split()[0])

In [10]:
classes = list(map(lambda x:get_class(TRAIN_LABELS_PATH / x), os.listdir(TRAIN_LABELS_PATH)))

In [11]:
def get_count(classes):
    count = dict()
    for i in classes:
        if names[i] not in count.keys():
            count[names[i]] = 0
        count[names[i]] += 1
    return dict(sorted(count.items(), key=lambda x:x[1], reverse=True))

In [12]:
count = get_count(classes)

In [13]:
count

{'Stop': 226,
 'Speed Limit 30': 196,
 'Speed Limit 80': 146,
 'Speed Limit 50': 134,
 'Speed Limit 70': 111,
 'Speed Limit 60': 88,
 'Turn Right': 64,
 'Speed Limit 100': 56,
 'No Entry': 42,
 'Give Way': 38,
 'Speed Limit 120': 38,
 'Left Sharp Curve': 36,
 'Right Sharp Curve': 33,
 'Danger Ahead': 24,
 'No Over Taking Trucks': 21,
 'No Over Taking': 19,
 'Road Work': 16,
 'Go Straight': 15,
 'Pedestrian': 13,
 'Slippery Road': 13,
 'Turn Left': 12,
 'Traffic Signals Ahead': 8,
 'Snow Warning Sign': 7,
 'Huddle Road': 6,
 'No Stopping': 5,
 'No Waiting': 4,
 'End of Right Road -Go straight-': 4,
 'Truck Sign': 4,
 'Go Right or Straight': 3,
 'Speed Limit 20': 3,
 'Right Curve Ahead': 2,
 'Go Left or Straight': 1,
 'Left Curve Ahead': 1,
 'RoundAbout': 1,
 'Cycle Zone': 1,
 'Deer Zone': 1}

In [14]:
VALID_LABELS_PATH = Path("../data/TrafficSignLocalizationandDetection/valid//labels/")
classes = list(map(lambda x:get_class(VALID_LABELS_PATH / x), os.listdir(VALID_LABELS_PATH)))
get_count(classes)

{'Stop': 31,
 'Speed Limit 30': 21,
 'Speed Limit 50': 18,
 'Speed Limit 70': 14,
 'Speed Limit 80': 14,
 'Speed Limit 60': 12,
 'Turn Right': 11,
 'No Entry': 8,
 'Speed Limit 100': 7,
 'Road Work': 6,
 'Danger Ahead': 6,
 'Give Way': 5,
 'Left Sharp Curve': 3,
 'Right Sharp Curve': 3,
 'Slippery Road': 2,
 'Right Curve Ahead': 2,
 'Speed Limit 120': 2,
 'Snow Warning Sign': 1,
 'Turn Left': 1,
 'Truck Sign': 1,
 'No Over Taking': 1,
 'Pedestrian': 1,
 'No Stopping': 1,
 'Huddle Road': 1,
 'Go Right or Straight': 1}

# Common utils

In [15]:
def display(img, coordinates):
    new_img = deepcopy(img)
    cv2.rectangle(new_img, 
                  (coordinates[0], coordinates[1]),
                  (coordinates[0] + coordinates[2], coordinates[1] + coordinates[3]),
                  (255,0,0),
                  1
                 )
    while True:
        cv2.imshow("img", new_img)
        if cv2.waitKey(0) & 0xFF == 27:
            cv2.destroyAllWindows()
            break

In [16]:
def convert_coordinates_to_yolo_annotation(coordinates, label, shape):
    dh, dw, _ = shape
    x,y,w,h = coordinates
    x_final = x + w if x + w < dw else dw
    y_final = y + h if y + h < dh else dh
    x_mean = (x + x_final) / (dw * 2)
    y_mean = (y + y_final) / (dh * 2)
    w_norm = (x_final - x) / dw
    h_norm = (y_final - y) / dh
    return [" ".join([str(label), str(x_mean), str(y_mean), str(w_norm), str(h_norm)])]

def convert_yolo_annotations_to_coordinates(annotation, shape):
    dh, dw, _ = shape
    _, x, y, w, h = [float(i) for i in annotation[0].strip().split()]
    x_start = (x - w / 2) * dw
    y_start = (y - h / 2) * dh
    return np.array([x_start, y_start, w * dw, h * dh], dtype=np.uint8)

In [17]:
def get_labels_from_file(path):
    with open(path, "r") as f:
        l = f.readlines()
    return l

# Augmentation 1 - Local Histogram Equalization

In [18]:
def local_histogram_equalization(img):
    new_img = np.zeros(img.shape, dtype=np.uint8)
    new_img[:,:,0] = cv2.equalizeHist(img[:,:,0])
    new_img[:,:,1] = cv2.equalizeHist(img[:,:,1])
    new_img[:,:,2] = cv2.equalizeHist(img[:,:,2])
    return new_img

In [19]:
img = cv2.imread("../data/TrafficSignLocalizationandDetection/train/images/000001_jpg.rf.4f8eac6747a989e40bff9c86bc5d97be.jpg")
labels = get_labels_from_file("../data/TrafficSignLocalizationandDetection/train/labels/000001_jpg.rf.4f8eac6747a989e40bff9c86bc5d97be.txt")

In [20]:
coordinates = convert_yolo_annotations_to_coordinates(labels, img.shape)

In [17]:
display(local_histogram_equalization(img), coordinates)

# Augmentation 2 - Changing Brightness

In [33]:
def change_brightness(img, low=0.3, high=10):
    value = np.random.uniform(low, high)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    hsv = np.array(hsv, dtype = np.float64)
    hsv[:,:,1] = hsv[:,:,1]*value
    hsv[:,:,1][hsv[:,:,1]>255]  = 255
    hsv[:,:,2] = hsv[:,:,2]*value 
    hsv[:,:,2][hsv[:,:,2]>255]  = 255
    hsv = np.array(hsv, dtype = np.uint8)
    img = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    return img

In [34]:
display(change_brightness(img), coordinates)

# Augmentation 3 - Rotating Image

In [24]:
def rotation_changing(image):
    # Defining angle range
    angle_range = 25
    # Defining angle rotation
    angle_rotation = np.random.uniform(angle_range) - angle_range / 2
    # Getting shape of image
    rows, columns, channels = image.shape
    # Implementing rotation
    # Calculating Affine Matrix
    affine_matrix = cv2.getRotationMatrix2D((columns / 2, rows / 2), angle_rotation, 1)
    # Warping original image with Affine Matrix
    rotated_image = cv2.warpAffine(image, affine_matrix, (columns, rows))
    # Returning rotated image
    return rotated_image

In [88]:
display(rotation_changing(img), coordinates)

# Preprocessing data